## Pseudocode Fenster (GUI)



- Fenster generieren
- Welche Buttons werden gebraucht?
- weitere interaktive Elemente (z.B. drop down, Suche)
- den Kalender (Tabelle)



In [1]:
import tkinter as tk
from tkinter import messagebox
from tkinter import ttk
import calendar
from datetime import datetime, timedelta

class CalendarApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Terminbuchung")

        # Styling
        self.root.configure(bg="#f0f0f0")

        # Aktuelles Datum
        self.current_date = datetime.now()

        # Erstelle Kalenderanzeige
        self.create_calendar(self.current_date)

    def create_calendar(self, date):
        """Erstellt den Kalender basierend auf dem Datum"""
        self.date_frame = tk.Frame(self.root, bg="#f0f0f0")
        self.date_frame.pack(pady=10)

        # Zeigt den Monat und Jahr an
        self.month_year_label = tk.Label(self.date_frame, text=date.strftime("%B %Y"), font=("Arial", 16), bg="#f0f0f0")
        self.month_year_label.grid(row=0, column=1)

        # Kalender (Matrix)
        self.calendar_frame = tk.Frame(self.root, bg="#f0f0f0")
        self.calendar_frame.pack()

        # Wochentage
        days = ["Mo", "Di", "Mi", "Do", "Fr", "Sa", "So"]
        for i, day in enumerate(days):
            tk.Label(self.calendar_frame, text=day, font=("Arial", 10), bg="#f0f0f0", fg="#333333").grid(row=0, column=i)

        # Kalender für den aktuellen Monat
        month_calendar = calendar.monthcalendar(date.year, date.month)

        # Datumsschaltflächen erstellen
        self.buttons = {}
        for row, week in enumerate(month_calendar, start=1):
            for col, day in enumerate(week):
                if day != 0:
                    button = tk.Button(self.calendar_frame, text=str(day), width=5, height=2, 
                                       command=lambda day=day: self.select_date(day), 
                                       font=("Arial", 12), fg="#333333", bg="#f8f8f8", 
                                       activebackground="#4CAF50", activeforeground="white")
                    button.grid(row=row, column=col, padx=5, pady=5)
                    # Aktuellen Tag hervorheben
                    if day == self.current_date.day and self.current_date.month == date.month:
                        button.config(bg="#4CAF50", fg="white")
                    self.buttons[day] = button

        # Button für den nächsten Monat
        self.next_button = tk.Button(self.date_frame, text=">>", command=self.next_month, font=("Arial", 14),
                                     bg="#4CAF50", fg="white", relief="solid", padx=10)
        self.next_button.grid(row=0, column=2)

        # Button für den vorherigen Monat
        self.prev_button = tk.Button(self.date_frame, text="<<", command=self.prev_month, font=("Arial", 14),
                                     bg="#4CAF50", fg="white", relief="solid", padx=10)
        self.prev_button.grid(row=0, column=0)

    def select_date(self, day):
        """Wird aufgerufen, wenn ein Datum aus dem Kalender ausgewählt wird"""
        self.selected_date = datetime(self.current_date.year, self.current_date.month, day)
        self.show_time_selector()

    def show_time_selector(self):
        """Zeigt eine Zeitwahl in 15-Minuten-Abständen in einer Tabelle an"""
        # Entferne vorherige Zeitauswahl (falls vorhanden)
        for widget in self.root.winfo_children():
            widget.destroy()

        # Zeigt das ausgewählte Datum an
        date_label = tk.Label(self.root, text=f"Ausgewähltes Datum: {self.selected_date.strftime('%d.%m.%Y')}",
                              font=("Arial", 14), bg="#f0f0f0", fg="#333333")
        date_label.pack(pady=10)

        # Zeigt die Tabelle für die Uhrzeiten in 15-Minuten-Abständen an
        self.time_frame = tk.Frame(self.root, bg="#f0f0f0")
        self.time_frame.pack()

        # Treeview für Uhrzeiten erstellen
        self.tree = ttk.Treeview(self.time_frame, columns=("Uhrzeit"), show="headings", height=10)
        self.tree.heading("Uhrzeit", text="Uhrzeit")
        self.tree.column("Uhrzeit", width=100, anchor="center")

        # Tabelleninhalt erstellen (15 Minuten Schritte)
        start_time = datetime(self.selected_date.year, self.selected_date.month, self.selected_date.day, 8, 0)
        end_time = datetime(self.selected_date.year, self.selected_date.month, self.selected_date.day, 18, 0)

        time_slots = []
        while start_time <= end_time:
            time_slots.append(start_time.strftime("%H:%M"))
            start_time += timedelta(minutes=15)

        # Zeilen in die Tabelle einfügen
        for time_slot in time_slots:
            self.tree.insert("", "end", values=(time_slot))

        # Button zum Termin buchen
        book_button = tk.Button(self.time_frame, text="Termin buchen", command=self.book_appointment, 
                                font=("Arial", 12), bg="#4CAF50", fg="white", activebackground="#45a049", activeforeground="white")
        book_button.pack(pady=10)

        # Pack Treeview
        self.tree.pack()

    def book_appointment(self):
        """Bucht den Termin mit Datum und Uhrzeit"""
        selected_item = self.tree.selection()
        if not selected_item:
            messagebox.showwarning("Auswahl erforderlich", "Bitte wählen Sie eine Uhrzeit aus!")
            return

        selected_time = self.tree.item(selected_item[0], "values")[0]
        appointment_time = f"{self.selected_date.strftime('%d.%m.%Y')} {selected_time}"
        messagebox.showinfo("Termin gebucht", f"Termin gebucht für {appointment_time}")

    def next_month(self):
        """Zeigt den nächsten Monat an"""
        self.current_date = self.current_date.replace(day=28) + timedelta(days=4)  # Gehe zum nächsten Monat
        self.refresh_calendar()

    def prev_month(self):
        """Zeigt den vorherigen Monat an"""
        self.current_date = self.current_date.replace(day=1) - timedelta(days=1)  # Gehe zum vorherigen Monat
        self.refresh_calendar()

    def refresh_calendar(self):
        """Aktualisiert den Kalender für den neuen Monat"""
        for widget in self.root.winfo_children():
            widget.destroy()
        self.create_calendar(self.current_date)

if __name__ == "__main__":
    root = tk.Tk()
    app = CalendarApp(root)
    root.mainloop()


In [2]:
import tkinter as tk
from tkinter import messagebox
from tkinter import ttk
import calendar
from datetime import datetime, timedelta

class CalendarApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Terminbuchung")

        # Styling
        self.root.configure(bg="#f0f0f0")

        # Aktuelles Datum
        self.current_date = datetime.now()

        # Erstelle Kalenderanzeige
        self.create_calendar(self.current_date)

    def create_calendar(self, date):
        """Erstellt den Kalender basierend auf dem Datum"""
        self.date_frame = tk.Frame(self.root, bg="#f0f0f0")
        self.date_frame.pack(pady=10)

        # Zeigt den Monat und Jahr an
        self.month_year_label = tk.Label(self.date_frame, text=date.strftime("%B %Y"), font=("Arial", 16), bg="#f0f0f0")
        self.month_year_label.grid(row=0, column=1)

        # Kalender (Matrix)
        self.calendar_frame = tk.Frame(self.root, bg="#f0f0f0")
        self.calendar_frame.pack()

        # Wochentage
        days = ["Mo", "Di", "Mi", "Do", "Fr", "Sa", "So"]
        for i, day in enumerate(days):
            tk.Label(self.calendar_frame, text=day, font=("Arial", 10), bg="#f0f0f0", fg="#333333").grid(row=0, column=i)

        # Kalender für den aktuellen Monat
        month_calendar = calendar.monthcalendar(date.year, date.month)

        # Datumsschaltflächen erstellen
        self.buttons = {}
        for row, week in enumerate(month_calendar, start=1):
            for col, day in enumerate(week):
                if day != 0:
                    button = tk.Button(self.calendar_frame, text=str(day), width=5, height=2, 
                                       command=lambda day=day: self.select_date(day), 
                                       font=("Arial", 12), fg="#333333", bg="#f8f8f8", 
                                       activebackground="#4CAF50", activeforeground="white")
                    button.grid(row=row, column=col, padx=5, pady=5)
                    # Aktuellen Tag hervorheben
                    if day == self.current_date.day and self.current_date.month == date.month:
                        button.config(bg="#4CAF50", fg="white")
                    self.buttons[day] = button

        # Button für den nächsten Monat
        self.next_button = tk.Button(self.date_frame, text=">>", command=self.next_month, font=("Arial", 14),
                                     bg="#4CAF50", fg="white", relief="solid", padx=10)
        self.next_button.grid(row=0, column=2)

        # Button für den vorherigen Monat
        self.prev_button = tk.Button(self.date_frame, text="<<", command=self.prev_month, font=("Arial", 14),
                                     bg="#4CAF50", fg="white", relief="solid", padx=10)
        self.prev_button.grid(row=0, column=0)

    def select_date(self, day):
        """Wird aufgerufen, wenn ein Datum aus dem Kalender ausgewählt wird"""
        self.selected_date = datetime(self.current_date.year, self.current_date.month, day)
        self.show_time_selector()

    def show_time_selector(self):
        """Zeigt eine Zeitwahl in 15-Minuten-Abständen in einer Tabelle an"""
        # Entferne vorherige Zeitauswahl (falls vorhanden)
        for widget in self.root.winfo_children():
            widget.destroy()

        # Zeigt das ausgewählte Datum an
        date_label = tk.Label(self.root, text=f"Ausgewähltes Datum: {self.selected_date.strftime('%d.%m.%Y')}",
                              font=("Arial", 14), bg="#f0f0f0", fg="#333333")
        date_label.pack(pady=10)

        # Zeigt die Tabelle für die Uhrzeiten in 15-Minuten-Abständen an
        self.time_frame = tk.Frame(self.root, bg="#f0f0f0")
        self.time_frame.pack()

        # Treeview für Uhrzeiten erstellen
        self.tree = ttk.Treeview(self.time_frame, columns=("Uhrzeit"), show="headings", height=10)
        self.tree.heading("Uhrzeit", text="Uhrzeit")
        self.tree.column("Uhrzeit", width=100, anchor="center")

        # Tabelleninhalt erstellen (15 Minuten Schritte)
        start_time = datetime(self.selected_date.year, self.selected_date.month, self.selected_date.day, 8, 0)
        end_time = datetime(self.selected_date.year, self.selected_date.month, self.selected_date.day, 18, 0)

        time_slots = []
        while start_time <= end_time:
            time_slots.append(start_time.strftime("%H:%M"))
            start_time += timedelta(minutes=15)

        # Zeilen in die Tabelle einfügen
        for time_slot in time_slots:
            self.tree.insert("", "end", values=(time_slot))

        # Button zum Termin buchen
        book_button = tk.Button(self.time_frame, text="Termin buchen", command=self.show_appointment_type_buttons, 
                                font=("Arial", 12), bg="#4CAF50", fg="white", activebackground="#45a049", activeforeground="white")
        book_button.pack(pady=10)

        # Pack Treeview
        self.tree.pack()

    def show_appointment_type_buttons(self):
        """Zeigt die Auswahl für den Termin-Typ (Beratungsgespräch, Probetraining, etc.)"""
        # Entferne vorherige Widgets
        for widget in self.root.winfo_children():
            widget.destroy()

        # Zeigt das ausgewählte Datum an
        date_label = tk.Label(self.root, text=f"Ausgewähltes Datum: {self.selected_date.strftime('%d.%m.%Y')}",
                              font=("Arial", 14), bg="#f0f0f0", fg="#333333")
        date_label.pack(pady=10)

        # Buttons für den Termin-Typ
        button_frame = tk.Frame(self.root, bg="#f0f0f0")
        button_frame.pack(pady=10)

        btn_beratung = tk.Button(button_frame, text="Beratungsgespräch", command=lambda: self.book_appointment("Beratungsgespräch"), 
                                  font=("Arial", 12), bg="#4CAF50", fg="white", activebackground="#45a049", activeforeground="white")
        btn_beratung.grid(row=0, column=0, padx=10, pady=5)

        btn_probetraining = tk.Button(button_frame, text="Probetraining", command=lambda: self.book_appointment("Probetraining"), 
                                      font=("Arial", 12), bg="#4CAF50", fg="white", activebackground="#45a049", activeforeground="white")
        btn_probetraining.grid(row=0, column=1, padx=10, pady=5)

        btn_training = tk.Button(button_frame, text="Training", command=lambda: self.book_appointment("Training"), 
                                  font=("Arial", 12), bg="#4CAF50", fg="white", activebackground="#45a049", activeforeground="white")
        btn_training.grid(row=1, column=0, padx=10, pady=5)

        btn_back = tk.Button(button_frame, text="Zurück", command=self.show_time_selector, 
                             font=("Arial", 12), bg="#f44336", fg="white", activebackground="#d32f2f", activeforeground="white")
        btn_back.grid(row=1, column=1, padx=10, pady=5)

    def book_appointment(self, appointment_type):
        """Bucht den Termin mit Datum und Uhrzeit"""
        selected_item = self.tree.selection()
        if not selected_item:
            messagebox.showwarning("Auswahl erforderlich", "Bitte wählen Sie eine Uhrzeit aus!")
            return

        selected_time = self.tree.item(selected_item[0], "values")[0]
        appointment_time = f"{self.selected_date.strftime('%d.%m.%Y')} {selected_time} ({appointment_type})"
        messagebox.showinfo("Termin gebucht", f"Termin gebucht für {appointment_time}")

    def next_month(self):
        """Zeigt den nächsten Monat an"""
        self.current_date = self.current_date.replace(day=28) + timedelta(days=4)  # Gehe zum nächsten Monat
        self.refresh_calendar()

    def prev_month(self):
        """Zeigt den vorherigen Monat an"""
        self.current_date = self.current_date.replace(day=1) - timedelta(days=1)  # Gehe zum vorherigen Monat
        self.refresh_calendar()

    def refresh_calendar(self):
        """Aktualisiert den Kalender für den neuen Monat"""
        for widget in self.root.winfo_children():
            widget.destroy()
        self.create_calendar(self.current_date)

if __name__ == "__main__":
    root = tk.Tk()
    app = CalendarApp(root)
    root.mainloop()
